In [ ]:
!pip install numpy==1.25.2
!pip uninstall scipy
!pip install scipy

In [ ]:
#imports

from google.colab import files
import pandas as pd
import numpy as np

In [ ]:
ACS_cleaned = pd.read_csv('/content/ACS_cleaned.csv')
Bikeways_cleaned = pd.read_csv('/content/Bikeways_cleaned.csv', dtype={'CENSUS_TRACT': str})
Crashes_cleaned = pd.read_csv('/content/Crashes_cleaned.csv', dtype={'CENSUS_TRACT': str})
Income_cleaned = pd.read_csv('/content/Income_cleaned.csv')

In [ ]:
print(Bikeways_cleaned)


     OBJECTID         CNN CORNER          STREET1         STREET2  \
0        2246  23875000.0      E           7TH ST       BRYANT ST   
1        4188  24570000.0     SE  THE EMBARCADERO        BROADWAY   
2          29  25474000.0    NaN  THE EMBARCADERO  NORTH POINT ST   
3        5475  34141000.0     NE       MACALLA RD  YERBA BUENA RD   
4        5133  20725000.0     NW        EVANS AVE     NAPOLEON ST   
..        ...         ...    ...              ...             ...   
487      6437  24017000.0     SE          17TH ST     POTRERO AVE   
488       199  25880000.0    NaN        MARKET ST         15TH ST   
489      4187  24570000.0    NaN  THE EMBARCADERO        BROADWAY   
490       133  24403000.0    NaN       DUBOCE AVE     VALENCIA ST   
491      6727  24262000.0    NaN           7TH ST       FOLSOM ST   

                 DESCRIPT  COUNT  INSTALL_MO  INSTALL_YR  UPDATE_MO  \
0             BIKE SIGNAL    2.0         4.0      2020.0        NaN   
1          TWO-STAGE LEFT    

In [ ]:
# 1. Get unique CENSUS_TRACT values from Bikeways and Crashes
all_census_tracts = pd.unique(pd.concat([Bikeways_cleaned['CENSUS_TRACT'], Crashes_cleaned['CENSUS_TRACT']]))

# 2. Get CENSUS_TRACT values from ACS_cleaned column labels (row 0)
acs_census_tracts = ACS_cleaned.columns[1:].tolist()  # Get column labels starting from index 1

# 3. Combine all unique CENSUS_TRACT values
all_census_tracts = pd.unique(all_census_tracts.tolist() + acs_census_tracts)

# 4. Create new DataFrame with single column
combined_df = pd.DataFrame({'CENSUS_TRACT': all_census_tracts})

# Display the new DataFrame
print(combined_df)


# Get column names from ACS_cleaned, ensuring they are strings
new_columns = ACS_cleaned.iloc[:3, 0].astype(str).values.tolist()

# Add new columns to combined_df with NaN values, ensuring column names are strings
for column_name in new_columns:
    combined_df[column_name] = np.nan  # Initialize with NaN

# Combine 'CROSSBIKE' and 'CROSS BIKE' columns
if 'CROSSBIKE' in combined_df.columns and 'CROSS BIKE' in combined_df.columns:
    combined_df['CROSSBIKE'] = combined_df['CROSSBIKE'].fillna(0) + combined_df['CROSS BIKE'].fillna(0)
    combined_df.drop('CROSS BIKE', axis=1, inplace=True)

# Display the updated combined_df
print(combined_df)


    CENSUS_TRACT
0    06075018000
1    06075010500
2    06075010101
3    06075017903
4    06075980900
..           ...
293  06075980401
294  06075009806
295  06075009809
296  06075009901
297  06075009902

[298 rows x 1 columns]
    CENSUS_TRACT  Total:      Bicycle  % Bike Commuter
0    06075018000     NaN          NaN              NaN
1    06075010500     NaN          NaN              NaN
2    06075010101     NaN          NaN              NaN
3    06075017903     NaN          NaN              NaN
4    06075980900     NaN          NaN              NaN
..           ...     ...          ...              ...
293  06075980401     NaN          NaN              NaN
294  06075009806     NaN          NaN              NaN
295  06075009809     NaN          NaN              NaN
296  06075009901     NaN          NaN              NaN
297  06075009902     NaN          NaN              NaN

[298 rows x 4 columns]


<ipython-input-36-111725e3b056>:8: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  all_census_tracts = pd.unique(all_census_tracts.tolist() + acs_census_tracts)


In [ ]:
# Transpose ACS_cleaned and reset index
acs_transposed = ACS_cleaned.set_index('Label (Grouping)').T.reset_index()
acs_transposed.rename(columns={'index': 'CENSUS_TRACT'}, inplace=True)

# Update combined_df using a loop and loc
for index in combined_df.index:
    census_tract = combined_df.loc[index, 'CENSUS_TRACT']

    # Find matching row in acs_transposed
    matching_row = acs_transposed[acs_transposed['CENSUS_TRACT'] == census_tract]

    # If a match is found, update values in combined_df
    if not matching_row.empty:
        for column in ACS_cleaned['Label (Grouping)'].tolist():
            if column in combined_df.columns:
                combined_df.loc[index, column] = matching_row[column].values[0]

# Display updated combined_df
print(combined_df)

    CENSUS_TRACT  Total:      Bicycle  % Bike Commuter
0    06075018000     NaN          NaN              NaN
1    06075010500     NaN          NaN              NaN
2    06075010101  1159.0         44.0         3.796376
3    06075017903  1397.0         21.0         1.503221
4    06075980900     NaN          NaN              NaN
..           ...     ...          ...              ...
293  06075980401     0.0          0.0         0.000000
294  06075009806   741.0         13.0         1.754386
295  06075009809   178.0         12.0         6.741573
296  06075009901     0.0          0.0         0.000000
297  06075009902     0.0          0.0         0.000000

[298 rows x 4 columns]


In [ ]:
# Delete rows with NaN in 'Total:' column
combined_df = combined_df.dropna(subset=['Total:'])

# Display the updated DataFrame
print(combined_df)

    CENSUS_TRACT  Total:      Bicycle  % Bike Commuter
2    06075010101  1159.0         44.0         3.796376
3    06075017903  1397.0         21.0         1.503221
6    06075016802  2726.0         40.0         1.467351
8    06075020202  2778.0        365.0        13.138949
9    06075016101   970.0          5.0         0.515464
..           ...     ...          ...              ...
293  06075980401     0.0          0.0         0.000000
294  06075009806   741.0         13.0         1.754386
295  06075009809   178.0         12.0         6.741573
296  06075009901     0.0          0.0         0.000000
297  06075009902     0.0          0.0         0.000000

[244 rows x 4 columns]


In [ ]:
combined_df['Num_Crashes'] = 0
print(combined_df)

    CENSUS_TRACT  Total:      Bicycle  % Bike Commuter  Num_Crashes
2    06075010101  1159.0         44.0         3.796376            0
3    06075017903  1397.0         21.0         1.503221            0
6    06075016802  2726.0         40.0         1.467351            0
8    06075020202  2778.0        365.0        13.138949            0
9    06075016101   970.0          5.0         0.515464            0
..           ...     ...          ...              ...          ...
293  06075980401     0.0          0.0         0.000000            0
294  06075009806   741.0         13.0         1.754386            0
295  06075009809   178.0         12.0         6.741573            0
296  06075009901     0.0          0.0         0.000000            0
297  06075009902     0.0          0.0         0.000000            0

[244 rows x 5 columns]


<ipython-input-39-d9d4ecead7d5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_df['Num_Crashes'] = 0


In [ ]:
for index, row in Crashes_cleaned.iterrows():
    census_tract = row['CENSUS_TRACT']

    # Find matching rows in combined_df
    matching_rows = combined_df[combined_df['CENSUS_TRACT'] == census_tract]

    # If a match is found, increment Num_Crashes
    if not matching_rows.empty:
        combined_df.loc[matching_rows.index, 'Num_Crashes'] += 1

In [ ]:
combined_df['COUNT'] = 0


In [ ]:
Bikeways_cleaned = Bikeways_cleaned[['DESCRIPT', 'COUNT', 'CENSUS_TRACT']]
print(Bikeways_cleaned)

                 DESCRIPT  COUNT CENSUS_TRACT
0             BIKE SIGNAL    2.0  06075018000
1          TWO-STAGE LEFT    1.0  06075010500
2    INTERSECTION SHARROW    2.0  06075010101
3                BIKE BOX    1.0  06075017903
4             MIXING ZONE    1.0  06075980900
..                    ...    ...          ...
487        TWO-STAGE LEFT    1.0  06075022802
488              BIKE BOX    1.0  06075020300
489             CROSSBIKE    1.0  06075010500
490           BIKE SIGNAL    4.0  06075020101
491           BIKE SIGNAL    2.0  06075017804

[492 rows x 3 columns]


In [ ]:
for item in Bikeways_cleaned['DESCRIPT'].unique():
    print(item)

BIKE SIGNAL
TWO-STAGE LEFT
INTERSECTION SHARROW
BIKE BOX
MIXING ZONE
PAINTED SAFETY ZONE
CROSSBIKE
PROTECTED CORNER
nan
CROSS BIKE
PROTECTED INTERSECTION
BIKE CHANNEL
JUGHANDLE


In [ ]:
#num_nan_rows = Bikeways_cleaned['DESCRIPT'].isna().sum()
#print(f"Number of rows with 'nan' in 'DESCRIPT': {num_nan_rows}")
#result- only one nan row

Bikeways_cleaned = Bikeways_cleaned.dropna(subset=['DESCRIPT'])


In [ ]:
for item in Bikeways_cleaned['DESCRIPT'].unique():
    print(item)

BIKE SIGNAL
TWO-STAGE LEFT
INTERSECTION SHARROW
BIKE BOX
MIXING ZONE
PAINTED SAFETY ZONE
CROSSBIKE
PROTECTED CORNER
CROSS BIKE
PROTECTED INTERSECTION
BIKE CHANNEL
JUGHANDLE


In [ ]:
# Get unique items from 'DESCRIPT'
unique_items = Bikeways_cleaned['DESCRIPT'].unique()

# Create a new DataFrame
item_df = pd.DataFrame({'DESCRIPT': unique_items})

# Display the new DataFrame
print(item_df)

                  DESCRIPT
0              BIKE SIGNAL
1           TWO-STAGE LEFT
2     INTERSECTION SHARROW
3                 BIKE BOX
4              MIXING ZONE
5      PAINTED SAFETY ZONE
6                CROSSBIKE
7         PROTECTED CORNER
8               CROSS BIKE
9   PROTECTED INTERSECTION
10            BIKE CHANNEL
11               JUGHANDLE


In [ ]:
# Create a dictionary mapping DESCRIPT values to row numbers
descript_mapping = {descript: i for i, descript in enumerate(item_df['DESCRIPT'])}

# Replace values in Bikeways_cleaned['DESCRIPT']
Bikeways_cleaned['DESCRIPT'] = Bikeways_cleaned['DESCRIPT'].map(descript_mapping)

# Display the updated Bikeways_cleaned DataFrame
print(Bikeways_cleaned)

     DESCRIPT  COUNT CENSUS_TRACT
0           0    2.0  06075018000
1           1    1.0  06075010500
2           2    2.0  06075010101
3           3    1.0  06075017903
4           4    1.0  06075980900
..        ...    ...          ...
487         1    1.0  06075022802
488         3    1.0  06075020300
489         6    1.0  06075010500
490         0    4.0  06075020101
491         0    2.0  06075017804

[491 rows x 3 columns]


In [ ]:
descript_counts = Bikeways_cleaned['DESCRIPT'].value_counts()
print(descript_counts)

DESCRIPT
2     166
0      74
1      71
7      46
4      45
6      41
3      40
5       3
11      2
8       1
9       1
10      1
Name: count, dtype: int64


In [ ]:
# Iterate through each row in item_df

item_df = item_df[item_df['DESCRIPT'] != 'CROSS BIKE']


for index, row in item_df.iterrows():
    descript = row['DESCRIPT']
    category_number = index  # Use row index as category number

    # Create column name
    column_name = f"{descript}: {category_number}"

    # Add new column to combined_df, initialized with 0
    combined_df[column_name] = 0

# Display the updated combined_df
print(combined_df.columns)

Index(['CENSUS_TRACT', 'Total:', '    Bicycle', '% Bike Commuter',
       'Num_Crashes', 'COUNT', 'BIKE SIGNAL: 0', 'TWO-STAGE LEFT: 1',
       'INTERSECTION SHARROW: 2', 'BIKE BOX: 3', 'MIXING ZONE: 4',
       'PAINTED SAFETY ZONE: 5', 'CROSSBIKE: 6', 'PROTECTED CORNER: 7',
       'PROTECTED INTERSECTION: 9', 'BIKE CHANNEL: 10', 'JUGHANDLE: 11'],
      dtype='object')


In [ ]:
for index, row in Bikeways_cleaned.iterrows():
    census_tract = row['CENSUS_TRACT']
    descript_category = row['DESCRIPT']
    count = row['COUNT']

    # Find matching row in combined_df
    matching_rows = combined_df[combined_df['CENSUS_TRACT'] == census_tract]

    # If a match is found, update the corresponding Descript column
    if not matching_rows.empty:
        # Construct column name using existing column format
        column_name = f"{item_df.loc[descript_category, 'DESCRIPT']}: {descript_category}"

        # Check if column exists in combined_df
        if column_name in combined_df.columns:
            combined_df.loc[matching_rows.index, column_name] += count

# Display the updated combined_df (optional)
print(combined_df)

    CENSUS_TRACT  Total:      Bicycle  % Bike Commuter  Num_Crashes  COUNT  \
2    06075010101  1159.0         44.0         3.796376            9      0   
3    06075017903  1397.0         21.0         1.503221            0      0   
6    06075016802  2726.0         40.0         1.467351            6      0   
8    06075020202  2778.0        365.0        13.138949            7      0   
9    06075016101   970.0          5.0         0.515464            1      0   
..           ...     ...          ...              ...          ...    ...   
293  06075980401     0.0          0.0         0.000000            0      0   
294  06075009806   741.0         13.0         1.754386            0      0   
295  06075009809   178.0         12.0         6.741573            0      0   
296  06075009901     0.0          0.0         0.000000            0      0   
297  06075009902     0.0          0.0         0.000000            0      0   

     BIKE SIGNAL: 0  TWO-STAGE LEFT: 1  INTERSECTION SHARROW: 2

In [ ]:
# Select the columns to sum
columns_to_sum = ['BIKE SIGNAL: 0', 'TWO-STAGE LEFT: 1',
       'INTERSECTION SHARROW: 2', 'BIKE BOX: 3', 'MIXING ZONE: 4',
       'PAINTED SAFETY ZONE: 5', 'CROSSBIKE: 6', 'PROTECTED CORNER: 7',
        'PROTECTED INTERSECTION: 9', 'BIKE CHANNEL: 10',
       'JUGHANDLE: 11']

# Calculate the sum for each row and store it in 'COUNT'
combined_df['COUNT'] = combined_df[columns_to_sum].sum(axis=1)

# Display the updated DataFrame (optional)
print(combined_df)

    CENSUS_TRACT  Total:      Bicycle  % Bike Commuter  Num_Crashes  COUNT  \
2    06075010101  1159.0         44.0         3.796376            9    8.0   
3    06075017903  1397.0         21.0         1.503221            0    5.0   
6    06075016802  2726.0         40.0         1.467351            6   11.0   
8    06075020202  2778.0        365.0        13.138949            7   31.0   
9    06075016101   970.0          5.0         0.515464            1    1.0   
..           ...     ...          ...              ...          ...    ...   
293  06075980401     0.0          0.0         0.000000            0    0.0   
294  06075009806   741.0         13.0         1.754386            0    0.0   
295  06075009809   178.0         12.0         6.741573            0    0.0   
296  06075009901     0.0          0.0         0.000000            0    0.0   
297  06075009902     0.0          0.0         0.000000            0    0.0   

     BIKE SIGNAL: 0  TWO-STAGE LEFT: 1  INTERSECTION SHARROW: 2

In [ ]:
combined_df['Relative_Crashes'] = np.where(combined_df['% Bike Commuter'] != 0,
                                               combined_df['Num_Crashes'] / combined_df['% Bike Commuter'],
                                               0)  # Assign 0 if '% Bike Commuter' is 0

In [ ]:
print(combined_df.columns)

Index(['CENSUS_TRACT', 'Total:', '    Bicycle', '% Bike Commuter',
       'Num_Crashes', 'COUNT', 'BIKE SIGNAL: 0', 'TWO-STAGE LEFT: 1',
       'INTERSECTION SHARROW: 2', 'BIKE BOX: 3', 'MIXING ZONE: 4',
       'PAINTED SAFETY ZONE: 5', 'CROSSBIKE: 6', 'PROTECTED CORNER: 7',
       'PROTECTED INTERSECTION: 9', 'BIKE CHANNEL: 10', 'JUGHANDLE: 11',
       'Relative_Crashes'],
      dtype='object')


In [ ]:
# 1. Ensure both columns are strings
combined_df['CENSUS_TRACT'] = combined_df['CENSUS_TRACT'].astype(str)
Income_cleaned['Census Tract'] = Income_cleaned['Census Tract'].astype(str)

# 2. Remove leading/trailing spaces
combined_df['CENSUS_TRACT'] = combined_df['CENSUS_TRACT'].str.strip()
Income_cleaned['Census Tract'] = Income_cleaned['Census Tract'].str.strip()

# 3. Pad 'Census Tract' in Income_cleaned with leading zeros to 11 characters
Income_cleaned['Census Tract'] = Income_cleaned['Census Tract'].str.zfill(11)


In [ ]:
# Merge the DataFrames based on 'CENSUS_TRACT' and 'Census Tract' columns
combined_df = pd.merge(combined_df, Income_cleaned[['Census Tract', 'Median Income']], left_on='CENSUS_TRACT', right_on='Census Tract', how='left')

# Drop the redundant 'Census Tract' column
combined_df.drop('Census Tract', axis=1, inplace=True)

# Display the merged DataFrame (optional)
print(combined_df.head())

  CENSUS_TRACT  Total:      Bicycle  % Bike Commuter  Num_Crashes  COUNT  \
0  06075010101  1159.0         44.0         3.796376            9    8.0   
1  06075017903  1397.0         21.0         1.503221            0    5.0   
2  06075016802  2726.0         40.0         1.467351            6   11.0   
3  06075020202  2778.0        365.0        13.138949            7   31.0   
4  06075016101   970.0          5.0         0.515464            1    1.0   

   BIKE SIGNAL: 0  TWO-STAGE LEFT: 1  INTERSECTION SHARROW: 2  BIKE BOX: 3  \
0               2                  1                        5            0   
1               0                  0                        2            1   
2               0                  1                        7            3   
3               0                  4                       19            5   
4               0                  0                        0            1   

   MIXING ZONE: 4  PAINTED SAFETY ZONE: 5  CROSSBIKE: 6  PROTECTED CORNER:

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Replace '-' with NaN in the DataFrame before scaling
combined_df = combined_df.replace('-', np.nan)

# Get all column names except 'CENSUS_TRACT'
cols_to_normalize = [col for col in combined_df.columns if col != 'CENSUS_TRACT']

# Convert all columns except 'CENSUS_TRACT' to numeric, coercing errors to NaN
for col in cols_to_normalize:
    combined_df[col] = pd.to_numeric(combined_df[col], errors='coerce')

# Create a MinMaxScaler object
scaler = MinMaxScaler()

# Fit the scaler to the selected columns
scaler.fit(combined_df[cols_to_normalize])

# Transform the selected columns and update the DataFrame
combined_df[cols_to_normalize] = scaler.transform(combined_df[cols_to_normalize])

# Display the normalized DataFrame (optional)
print(combined_df.head())

  CENSUS_TRACT    Total:      Bicycle  % Bike Commuter  Num_Crashes     COUNT  \
0  06075010101  0.210996     0.086614         0.224949     0.818182  0.205128   
1  06075017903  0.254324     0.041339         0.089071     0.000000  0.128205   
2  06075016802  0.496268     0.078740         0.086946     0.545455  0.282051   
3  06075020202  0.505735     0.718504         0.778532     0.636364  0.794872   
4  06075016101  0.176588     0.009843         0.030543     0.090909  0.025641   

   BIKE SIGNAL: 0  TWO-STAGE LEFT: 1  INTERSECTION SHARROW: 2  BIKE BOX: 3  \
0        0.166667                0.2                 0.263158          0.0   
1        0.000000                0.0                 0.105263          0.2   
2        0.000000                0.2                 0.368421          0.6   
3        0.000000                0.8                 1.000000          1.0   
4        0.000000                0.0                 0.000000          0.2   

   MIXING ZONE: 4  PAINTED SAFETY ZONE: 5  C

In [ ]:
# Find rows with any NaN values
nan_rows = combined_df[combined_df.isna().any(axis=1)]

# Display the rows with NaN values
print(nan_rows)

    CENSUS_TRACT    Total:      Bicycle  % Bike Commuter  Num_Crashes  \
5    06075017804  0.510650     0.108268         0.116184     0.454545   
104  06075000103  0.449117     0.120079         0.146513     0.000000   
106  06075000105  0.297834     0.104331         0.191959     0.000000   
107  06075000106  0.360277     0.031496         0.047906     0.000000   
109  06075000108  0.549791     0.114173         0.113799     0.000000   
..           ...       ...          ...              ...          ...   
239  06075980401  0.000000     0.000000         0.000000     0.000000   
240  06075009806  0.134899     0.025591         0.103954     0.000000   
241  06075009809  0.032405     0.023622         0.399463     0.000000   
242  06075009901  0.000000     0.000000         0.000000     0.000000   
243  06075009902  0.000000     0.000000         0.000000     0.000000   

        COUNT  BIKE SIGNAL: 0  TWO-STAGE LEFT: 1  INTERSECTION SHARROW: 2  \
5    0.692308        0.333333                1

In [ ]:
combined_df.to_csv('combined_df.csv', index=False)
files.download('combined_df.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>